In [2]:
import argparse
import os
import shutil
import sys
import time
import warnings
from random import sample

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiStepLR

# from cgcnn.data import CIFData
# from cgcnn.data import collate_pool, get_train_val_test_loader
from cgcnn.featureModel import CrystalGraphConvNet
from property_prediction_ofm.model import Net

from dataloader import CIFOFMData
from dataloader import collate_pool, get_train_val_test_loader

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
# Model
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class MixtureNet(nn.Module):
    def __init__(self, input_size=64):
        super(MixtureNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 48)
        self.fc2 = nn.Linear(48, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
mixnet = MixtureNet()
mixnet = mixnet.float()
mixnet.to(device)

MixtureNet(
  (fc1): Linear(in_features=64, out_features=48, bias=True)
  (fc2): Linear(in_features=48, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)

In [6]:
data_options = '../project_mlns/dataset/'
modelpathCG = 'trained_nets/formation-energy-per-atom.pth.tar'
modelpathOFM = 'trained_nets/propertyPredictionUsingOFM_net.pth'

workers = 0
epochs = 30
start_epoch = 0
batch_size = 32
lr = 0.01
lr_milestones = [100]
disable_cuda = False

momentum = 0.9
weight_decay = 0
print_freq = 10

val_ratio = 0.1
test_ratio = 0.1
optim_type ='SGD'

cuda = not disable_cuda and torch.cuda.is_available()

best_mae_error = 1e10

In [7]:
# load data
dataset = CIFOFMData(data_options)
collate_fn = collate_pool
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset=dataset,
    collate_fn=collate_fn,
    pin_memory=cuda,
    batch_size=batch_size,
    num_workers=workers,
    train_size = None,
    test_size = None,
    val_size = None,
    val_ratio=val_ratio,
    test_ratio=test_ratio,
    return_test=True, shuffle=True)

[Warning] train_ratio is None, using all training data.


In [8]:
class Normalizer(object):
    """Normalize a Tensor and restore it later. """

    def __init__(self, tensor):
        """tensor is taken as a sample to calculate the mean and std"""
        self.mean = torch.mean(tensor)
        self.std = torch.std(tensor)

    def norm(self, tensor):
        return (tensor - self.mean) / self.std

    def denorm(self, normed_tensor):
        return normed_tensor * self.std + self.mean

    def state_dict(self):
        return {'mean': self.mean,
                'std': self.std}

    def load_state_dict(self, state_dict):
        self.mean = state_dict['mean']
        self.std = state_dict['std']

In [10]:
if len(dataset) < 500:
    warnings.warn('Dataset has less than 500 data points. '
                  'Lower accuracy is expected. ')
    sample_data_list = [dataset[i] for i in range(len(dataset))]
else:
    sample_data_list = [dataset[i] for i in
                        sample(range(len(dataset)), 500)]
_, sample_target, _, _ = collate_pool(sample_data_list)
normalizer = Normalizer(sample_target)

# build models
structures, _, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

model_checkpoint = torch.load(modelpathCG,
                              map_location=lambda storage, loc: storage)
model_args = argparse.Namespace(**model_checkpoint['args'])
modelCG = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len,
                            atom_fea_len=model_args.atom_fea_len,
                            n_conv=model_args.n_conv,
                            h_fea_len=model_args.h_fea_len,
                            n_h=model_args.n_h,
                            classification=False)

modelOFM = Net()
modelOFM.load_state_dict(torch.load(modelpathOFM))

if cuda:
    modelOFM.cuda()
    modelCG.cuda()

In [12]:
# define loss func and optimizer
criterion = nn.MSELoss()
if optim_type == 'SGD':
    optimizer = optim.SGD(mixnet.parameters(), lr,
                          momentum=momentum,
                          weight_decay=weight_decay)
elif optim_type == 'Adam':
    optimizer = optim.Adam(mixnet.parameters(), lr,
                           weight_decay=weight_decay)
else:
    raise NameError('Only SGD or Adam is allowed as --optim')

In [13]:
# resume from a checkpoint
checkpointCG = torch.load(modelpathCG)
modelCG.load_state_dict(checkpointCG['state_dict'])
normalizer.load_state_dict(checkpointCG['normalizer'])

checkpointOFM = torch.load(modelpathOFM)
modelOFM.load_state_dict(checkpointOFM)

<All keys matched successfully>

In [14]:
def mae(prediction, target):
    """
    Computes the mean absolute error between prediction and target

    Parameters
    ----------

    prediction: torch.Tensor (N, 1)
    target: torch.Tensor (N, 1)
    """
    return torch.mean(torch.abs(target - prediction))


def class_eval(prediction, target):
    prediction = np.exp(prediction.numpy())
    target = target.numpy()
    pred_label = np.argmax(prediction, axis=1)
    target_label = np.squeeze(target)
    if not target_label.shape:
        target_label = np.asarray([target_label])
    if prediction.shape[1] == 2:
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            target_label, pred_label, average='binary')
        auc_score = metrics.roc_auc_score(target_label, prediction[:, 1])
        accuracy = metrics.accuracy_score(target_label, pred_label)
    else:
        raise NotImplementedError
    return accuracy, precision, recall, fscore, auc_score


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch, k):
    """Sets the learning rate to the initial LR decayed by 10 every k epochs"""
    assert type(k) is int
    lr = args.lr * (0.1 ** (epoch // k))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [19]:
def train(train_loader, mixnet, modelCG, modelOFM, criterion, optimizer, epoch, normalizer):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    mae_errors = AverageMeter()

    # switch to train mode
    mixnet.train()

    end = time.time()
    for i, (input, target, _, ofmMat) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if cuda:
            input_var = (Variable(input[0].cuda(non_blocking=True)),
                         Variable(input[1].cuda(non_blocking=True)),
                         input[2].cuda(non_blocking=True),
                         [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]])
            inOfmMat = ofmMat.float().cuda(non_blocking=True)
        else:
            input_var = (Variable(input[0]),
                         Variable(input[1]),
                         input[2],
                         input[3])
            inOfmMat = ofmMat.float()
            
        # normalize target
        target_normed = normalizer.norm(target)
        
        if cuda:
            target_var = Variable(target_normed.cuda(non_blocking=True))
        else:
            target_var = Variable(target_normed)

        # compute feature from CG and OFM models
        featureCG = modelCG(*input_var)
        featureOFM = modelOFM(inOfmMat)
        # print(featureCG.size(), featureOFM.size())   
        # final feature after concatenation of features from CG and OFM models
        feature = torch.cat((featureCG, featureOFM), 1)
        # print(feature.size())
        MAE = min_mae
        output = mixnet(feature)
        
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        mae_error = mae(normalizer.denorm(output.data.cpu()), target)
        losses.update(loss.data.cpu(), target.size(0))
        mae_errors.update(mae_error, target.size(0))
       

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if i % print_freq == 0:
#             print('Epoch: [{0}][{1}/{2}]\t'
#                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                   'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
#                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                   'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
#                 epoch, i, len(train_loader), batch_time=batch_time,
#                 data_time=data_time, loss=losses, mae_errors=mae_errors)
#             )


In [24]:
scheduler = MultiStepLR(optimizer, milestones=lr_milestones,
                        gamma=0.1)

for epoch in range(start_epoch, epochs):
    # train for one epoch
    train(train_loader, mixnet, modelCG, modelOFM, criterion, optimizer, epoch, normalizer)

    scheduler.step()
print("Final MAE: {}".format(MAE))

Final MAE: 0.041
